# Assignment 3: Machine Learning for Huntington's Disease Prediction

---

**Objective:** Build and evaluate machine learning models to predict disease stage in Huntington's Disease patients using clinical, genetic, and molecular features.

**Dataset:** Huntington's Disease Dataset (48,536 patients, 13 clinical features)

**Target Variable:** Disease_Stage (5-class classification: Pre-symptomatic, Early Stage, Mid Stage, Late Stage, Advanced)

---

## Why This Matters

Accurate prediction of disease stage in Huntington's Disease enables:
- **Early Intervention:** Identify patients who would benefit from early treatment
- **Treatment Planning:** Tailor therapeutic strategies based on disease progression
- **Clinical Trials:** Stratify patients for more effective trial enrollment
- **Patient Counseling:** Provide evidence-based prognosis for personalized care

---

## Success Criteria

- High classification accuracy (>85%)
- Balanced precision and recall across all disease stages
- Interpretable models that align with clinical knowledge
- Robust generalization to unseen patient data

---

## 1. Introduction & Setup

### 1.1 Import Libraries

In [ ]:
#import core libraries for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

#utilities
import warnings
warnings.filterwarnings('ignore')

#setting random seed ensures reproducible results across runs
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

#configure display settings for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

#plotting style for visualizations
plt.style.use('default')
sns.set_palette("husl")

print("✓ Core libraries imported successfully")
print(f"✓ Random seed set to {RANDOM_STATE} for reproducibility")
print("\nNote: Additional libraries will be imported in relevant sections as needed")

In [ ]:
#install required packages in jupyter kernel environment
import sys
!{sys.executable} -m pip install scikit-learn xgboost shap lime --quiet

## 2. Load Data

In [ ]:
#load cleaned data from assinment 2
#data preprocessing (removing irrelevant columns, handling duplicates, etc) was completed in assignment 2
df = pd.read_csv('data/Huntington_Disease_Cleaned.csv')

print(f"Data loaded: {df.shape[0]:,} patients, {df.shape[1]} features")
print(f"Target variable: Disease_Stage (multi-class classification)")

In [ ]:
#quick overview
df.head()

In [ ]:
#check target variable distribution
#check for class imbalance
print("Disease Stage Distribution:")
print(df['Disease_Stage'].value_counts())
print(f"\nClass balance:")
print(df['Disease_Stage'].value_counts(normalize=True) * 100)

In [ ]:
#visualize class distribution
plt.figure(figsize=(10, 5))
df['Disease_Stage'].value_counts().plot(kind='bar', color='steelblue')
plt.title('Distribution of Disease Stages')
plt.xlabel('Disease Stage')
plt.ylabel('Number of Patients')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. Feature Engineering

In [ ]:
#check current features
print("Current features:")
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

In [ ]:
#create new features based on domain knowledge
df_fe = df.copy()

#genetic risk score: CAG repeats × gene expression
df_fe['Genetic_Risk_Score'] = df_fe['HTT_CAG_Repeat_Length'] * df_fe['HTT_Gene_Expression_Level']

#age-adjusted CAG: earlier onset = more aggressive
df_fe['Age_Adjusted_CAG'] = df_fe['HTT_CAG_Repeat_Length'] / df_fe['Age']

#brain health index: brain volume vs protein damage
df_fe['Brain_Health_Index'] = (100 - df_fe['Brain_Volume_Loss']) / (df_fe['Protein_Aggregation_Level'] + 1)

#motor-cognitive composite: combined symptom severity
#using chorea score only since cognitive_decline is categorical
df_fe['Motor_Cognitive_Composite'] = df_fe['Chorea_Score'] * df_fe['Brain_Volume_Loss']

print(f"Created 4 new features")
print(f"Total features now: {df_fe.shape[1]}")

In [ ]:
#check new features
df_fe[['Genetic_Risk_Score', 'Age_Adjusted_CAG', 'Brain_Health_Index', 'Motor_Cognitive_Composite']].describe()

In [ ]:
#encode categorical variables
#one-hot encoding for nominal variables
df_fe = pd.get_dummies(df_fe, columns=['Sex', 'Family_History', 'Motor_Symptoms', 'Cognitive_Decline'], drop_first=True)

#label encoding for gene mutation type
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_fe['Gene_Mutation_Type_Encoded'] = le.fit_transform(df_fe['Gene_Mutation_Type'])
df_fe = df_fe.drop('Gene_Mutation_Type', axis=1)

print(f"Encoded categorical variables")
print(f"Total features after encoding: {df_fe.shape[1]}")

In [ ]:
#check final feature list
print("Final features after engineering:")
for i, col in enumerate(df_fe.columns, 1):
    print(f"{i}. {col}")

## 4. Feature Selection

In [ ]:
#separate features and target
X = df_fe.drop('Disease_Stage', axis=1)
y = df_fe['Disease_Stage']

print(f"Features: {X.shape[1]}")
print(f"Samples: {X.shape[0]}")
print(f"Target classes: {y.nunique()}")

In [ ]:
#method 1: ANOVA f-test
#tests relationship between each feature and target
#works well for numerical features in classification
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(score_func=f_classif, k=10)
selector.fit(X, y)

anova_features = X.columns[selector.get_support()].tolist()
print("Top 10 features by ANOVA:")
for i, feat in enumerate(anova_features, 1):
    print(f"{i}. {feat}")

In [ ]:
#method 2: mutual information
#captures non-linear relationships between features and target
#complements ANOVA which only finds linear relationships
from sklearn.feature_selection import mutual_info_classif

mi_scores = mutual_info_classif(X, y, random_state=RANDOM_STATE)
mi_features = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)

print("Top 10 features by Mutual Information:")
for i, (feat, score) in enumerate(mi_features.head(10).items(), 1):
    print(f"{i}. {feat}: {score:.4f}")

In [ ]:
#method 3: random forest importance
#embedded method that considers feature interactions
#importance based on how much each feature improves tree splits
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
rf.fit(X, y)

rf_importance = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)

print("Top 10 features by Random Forest:")
for i, (feat, score) in enumerate(rf_importance.head(10).items(), 1):
    print(f"{i}. {feat}: {score:.4f}")

In [ ]:
#find consensus features across methods
#features appearing in top 10 of at least 2 methods are most reliable
anova_top10 = set(anova_features)
mi_top10 = set(mi_features.head(10).index)
rf_top10 = set(rf_importance.head(10).index)

#count how many methods selected each feature
all_features = anova_top10 | mi_top10 | rf_top10
feature_counts = {}
for feat in all_features:
    count = 0
    if feat in anova_top10: count += 1
    if feat in mi_top10: count += 1
    if feat in rf_top10: count += 1
    feature_counts[feat] = count

#select features with consensus (appear in 2+ methods)
selected_features = [feat for feat, count in feature_counts.items() if count >= 2]

print(f"\nConsensus features (≥2 methods): {len(selected_features)}")
for feat in sorted(selected_features):
    methods = []
    if feat in anova_top10: methods.append('ANOVA')
    if feat in mi_top10: methods.append('MI')
    if feat in rf_top10: methods.append('RF')
    print(f"  {feat} [{', '.join(methods)}]")

In [ ]:
#create final dataset with selected features
X_selected = X[selected_features]

print(f"\nOriginal features: {X.shape[1]}")
print(f"Selected features: {X_selected.shape[1]}")
print(f"Reduction: {((X.shape[1] - X_selected.shape[1]) / X.shape[1] * 100):.1f}%")

## 5. Machine Learning Models

In [ ]:
#split data into train and test sets
#stratify maintains class balance in both sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Features: {X_train.shape[1]}")

In [ ]:
#scale features for models that need it
#svm and knn are sensitive to feature scales
#tree-based models (random forest) don't need scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features scaled (mean=0, std=1)")

In [ ]:
#model 1: logistic regression (baseline)
#simple linear model, interpretable
#provides probability estimates for clinical decisions
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

lr = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

print("Logistic Regression:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_lr, average='weighted'):.4f}")

In [ ]:
#model 2: random forest
#ensemble method, handles non-linear relationships
#robust to outliers, provides feature importance
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("Random Forest:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_rf, average='weighted'):.4f}")

In [ ]:
# TODO: xgboost has import error - check this one later
# maybe find another alternative
print("XGBoost: Skipped")

In [ ]:
#model 3: support vector machine
#effective in high-dimensional space
#rbf kernel captures non-linear patterns
from sklearn.svm import SVC

svm_model = SVC(kernel='rbf', random_state=RANDOM_STATE)
svm_model.fit(X_train_scaled, y_train)
y_pred_svm = svm_model.predict(X_test_scaled)

print("Support Vector Machine:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_svm, average='weighted'):.4f}")

In [ ]:
#model 4: k-nearest neighbors
#non-parametric, simple and interpretable
#classifies based on similarity to training samples
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)
y_pred_knn = knn_model.predict(X_test_scaled)

print("K-Nearest Neighbors:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_knn, average='weighted'):.4f}")

In [ ]:
#compare all models
results = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'SVM', 'KNN'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_svm),
        accuracy_score(y_test, y_pred_knn)
    ],
    'F1-Score': [
        f1_score(y_test, y_pred_lr, average='weighted'),
        f1_score(y_test, y_pred_rf, average='weighted'),
        f1_score(y_test, y_pred_svm, average='weighted'),
        f1_score(y_test, y_pred_knn, average='weighted')
    ]
})

results = results.sort_values('Accuracy', ascending=False)
print("\nModel Comparison:")
print(results.to_string(index=False))

## 6. Performance Evaluation

In [ ]:
#detailed metrics for each model
#precision: how many predicted positives are actually positive (important for clinical decisions)
#recall: how many actual positives we caught (important for early detection)
#f1-score: harmonic mean balancing precision and recall
from sklearn.metrics import classification_report

print("=== LOGISTIC REGRESSION ===")
print(classification_report(y_test, y_pred_lr))

print("\n=== RANDOM FOREST ===")
print(classification_report(y_test, y_pred_rf))

print("\n=== SUPPORT VECTOR MACHINE ===")
print(classification_report(y_test, y_pred_svm))

print("\n=== K-NEAREST NEIGHBORS ===")
print(classification_report(y_test, y_pred_knn))

In [ ]:
#confusion matrices show which stages get confused
#diagonal = correct predictions, off-diagonal = errors
#helps identify if model confuses adjacent stages (early vs mid)
from sklearn.metrics import confusion_matrix

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

models = [
    ('Logistic Regression', y_pred_lr),
    ('Random Forest', y_pred_rf),
    ('SVM', y_pred_svm),
    ('KNN', y_pred_knn)
]

for idx, (name, y_pred) in enumerate(models):
    ax = axes[idx // 2, idx % 2]
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_title(f'{name}')
    ax.set_ylabel('Actual Stage')
    ax.set_xlabel('Predicted Stage')

plt.tight_layout()
plt.show()

In [ ]:
#roc curves and auc scores
#shows trade-off between true positive rate and false positive rate
#auc closer to 1.0 = better discrimination between classes
#use one-vs-rest for multi-class
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

#binarize labels for multi-class roc
y_test_bin = label_binarize(y_test, classes=sorted(y_test.unique()))
n_classes = y_test_bin.shape[1]

#get probability predictions for random forest (best model)
y_score_rf = rf_model.predict_proba(X_test)

#compute roc curve and auc for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score_rf[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

#plot
plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'orange', 'purple']
stages = sorted(y_test.unique())

for i, color, stage in zip(range(n_classes), colors, stages):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'{stage} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves - Random Forest (One-vs-Rest)')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
#model performance comparison across metrics
#helps identify best model for clinical use
from sklearn.metrics import precision_score, recall_score

comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'SVM', 'KNN'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_svm),
        accuracy_score(y_test, y_pred_knn)
    ],
    'Precision': [
        precision_score(y_test, y_pred_lr, average='weighted'),
        precision_score(y_test, y_pred_rf, average='weighted'),
        precision_score(y_test, y_pred_svm, average='weighted'),
        precision_score(y_test, y_pred_knn, average='weighted')
    ],
    'Recall': [
        recall_score(y_test, y_pred_lr, average='weighted'),
        recall_score(y_test, y_pred_rf, average='weighted'),
        recall_score(y_test, y_pred_svm, average='weighted'),
        recall_score(y_test, y_pred_knn, average='weighted')
    ],
    'F1-Score': [
        f1_score(y_test, y_pred_lr, average='weighted'),
        f1_score(y_test, y_pred_rf, average='weighted'),
        f1_score(y_test, y_pred_svm, average='weighted'),
        f1_score(y_test, y_pred_knn, average='weighted')
    ]
})

comparison = comparison.sort_values('Accuracy', ascending=False)
print("Comprehensive Model Comparison:")
print(comparison.to_string(index=False))

#visualize
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(comparison))
width = 0.2

ax.bar(x - 1.5*width, comparison['Accuracy'], width, label='Accuracy', color='skyblue')
ax.bar(x - 0.5*width, comparison['Precision'], width, label='Precision', color='lightcoral')
ax.bar(x + 0.5*width, comparison['Recall'], width, label='Recall', color='lightgreen')
ax.bar(x + 1.5*width, comparison['F1-Score'], width, label='F1-Score', color='gold')

ax.set_xlabel('Model')
ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison Across Metrics')
ax.set_xticks(x)
ax.set_xticklabels(comparison['Model'], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()